In [1]:
import numpy as np
from random import random
from scipy.optimize import minimize

from qiskit import *
from qiskit.circuit.library.standard_gates import U2Gate
from qiskit.aqua.operators import WeightedPauliOperator
from qiskit.aqua.algorithms import NumPyEigensolver

In [43]:
def hamiltonian_operator(a):
    pauli_dict = {
        'paulis': [{"coeff":{'imag': 0.0, "real": -1.0}, "label":"Z"}]
    }
    return WeightedPauliOperator.from_dict(pauli_dict)

In [45]:
H = hamiltonian_operator('a')
exact_result = NumPyEigensolver(H).run()
reference_energy = min(np.real(exact_result.eigenvalues))
print('The exact ground state energy is: {}'.format(reference_energy))

The exact ground state energy is: -1.0


In [46]:
def quantum_state_preparation(circuit, parameters):
    q = circuit.qregs[0] # q is the quantum register where the info about qubits is stored
    circuit.rx(parameters[0], q[0]) # q[0] is our one and only qubit XD
    circuit.ry(parameters[1], q[0])
    return circuit

In [47]:
def vqe_circuit(parameters, measure):
    """
    Creates a device ansatz circuit for optimization.
    :param parameters_array: list of parameters for constructing ansatz state that should be optimized.
    :param measure: measurement type. E.g. 'Z' stands for Z measurement.
    :return: quantum circuit.
    """
    q = QuantumRegister(1)
    c = ClassicalRegister(1)
    circuit = QuantumCircuit(q, c)

    # quantum state preparation
    circuit = quantum_state_preparation(circuit, parameters)

    # measurement
    if measure == 'Z':
        circuit.measure(q[0], c[0])
    
    return circuit

In [52]:
def quantum_module(parameters, measure):
    # measure
    if measure == 'Z':
        circuit = vqe_circuit(parameters, 'Z')
    else:
        raise ValueError('Not valid input for measurement: input should be "I" or "X" or "Z" or "Y"')
    
    shots = 1024
    backend = BasicAer.get_backend('qasm_simulator')
    job = execute(circuit, backend, shots=shots)
    result = job.result()
    counts = result.get_counts()
    
    # expectation value estimation from counts
    expectation_value = 0
    for measure_result in counts:
        sign = -1
        if measure_result == '1':
            sign = +1
        expectation_value += sign * counts[measure_result] / shots
        
    return expectation_value

In [53]:
def pauli_operator_to_dict(pauli_operator):
   
    d = pauli_operator.to_dict()
    paulis = d['paulis']
    paulis_dict = {}

    for x in paulis:
        label = x['label']
        coeff = x['coeff']['real']
        paulis_dict[label] = coeff

    return paulis_dict
pauli_dict = pauli_operator_to_dict(H)

In [54]:
def vqe(parameters):
        
    # quantum_modules
   
    quantum_module_Z = pauli_dict['Z'] * quantum_module(parameters, 'Z')
    
    
    # summing the measurement results
    classical_adder =   quantum_module_Z
    
    return classical_adder

In [55]:
parameters_array = np.array([np.pi, np.pi])
tol = 1e-3 # tolerance for optimization precision.

vqe_result = minimize(vqe, parameters_array, method="Powell", tol=tol)
print('The exact ground state energy is: {}'.format(reference_energy))
print('The estimated ground state energy from VQE algorithm is: {}'.format(vqe_result.fun))

The exact ground state energy is: -1.0
The estimated ground state energy from VQE algorithm is: -1.0
